In [ ]:
%matplotlib notebook

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import gsw
from neutral_surfaces import pot_dens_surf

In [ ]:
OCCA_dir = "~/work/data/OCCA/"
xrs = xr.open_dataset(OCCA_dir + "DDsalt.0406annclim.nc")
xrt = xr.open_dataset(OCCA_dir + "DDtheta.0406annclim.nc")

In [ ]:
xrst = xr.merge((xrs, xrt))
xrst = xrst.assign(p=gsw.p_from_z(-xrst.Depth_c, 30))

In [ ]:
xrst

In [ ]:
%%timeit
s, t, p = pot_dens_surf(xrst.salt.values, 
                        xrst.theta.values, 
                        xrst.p.values, 
                        0, 1026, axis=1, tol=1e-4)

In [ ]:
# Here we discard the unneeded time axis.
s, t, p = pot_dens_surf(xrst.salt.values[0], 
                        xrst.theta.values[0], 
                        xrst.p.values, 
                        0, 1026, axis=0, tol=1e-4)

In [ ]:
fig, ax = plt.subplots()
cs = ax.contourf(xrst.Longitude_t.values, xrst.Latitude_t.values, p)
cbar = fig.colorbar(cs, ax=ax)
cbar.set_label("Pressure [dbar]")
ax.set_title(r"Depth of $\sigma_\theta = 26$ in OCCA")